# Variational and Approximate GPs 

变分和近似高斯过程用于各种情况:
- 当GP似然是非高斯时 (例如用于分类)。
- 扩大GP回归 (通过使用随机优化)。
- 将GPs作为更大概率模型的一部分。

使用GPyTorch可以实现各种类型的近似GP模型。所有近似模型都由以下3个可组合对象组成:
- `VariationalDistribution`，定义了后验近似诱导值的形式 $q(\boldsymbol u)$
- `VariationalStrategies`，定义如何通过 $q(\boldsymbol u)$ 计算 $q(\boldsymbol f_X)$
- `_ApproximateMarginalLogLikelihood`，定义学习近似后验的目标函数 (e.g. variational ELBO)

## Stochastic Variational GP Regression

In [1]:
import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

In [2]:
import urllib.request
import os
from scipy.io import loadmat
from math import floor

In [3]:
# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)

if not smoke_test and not os.path.isfile('./datasets/elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk',
                               './datasets/elevators.mat')  # 这里 mat 文件需要手动从链接中下载

if smoke_test:  # this is for running the notebook in our testing framework
    X, y = torch.randn(1000, 3), torch.randn(1000)
else:
    data = torch.Tensor(loadmat('./datasets/elevators.mat')['data'])
    X = data[:, :-1]
    X = X - X.min(0)[0]
    X = 2 * (X / X.max(0)[0]) - 1
    y = data[:, -1]

train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
# if torch.cuda.is_available():
# train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()
train_x, train_y, test_x, test_y = train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)


### Creating a DataLoader

下一步是创建一个 `DataLoader`，它将处理随机的小批量数据。这涉及到使用PyTorch提供的标准 `TensorDataset` 和 `DataLoader` 模块。

In [4]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)  # original: batch_size=1024

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

### Creating a SVGP Model

对于大多数变分/近似GP模型，需要构造以下GPyTorch对象:
- 一个 **GP模型** (<font color=red>`gpytorch.models.ApproximateGP`</font>) - 处理基本的变分推断。
- Variational distribution (<font color=red>`gpytorch.variational._VariationalDistribution`</font>) - 这告诉我们 $q(\boldsymbol u)$ 的变分分布应该是什么形式。
- Variational strategy (<font color=red> `gpytorch.variational._VariationalStrategy` </font>) - 这告诉我们如何将 $q(\boldsymbol u)$ 在诱导点值上的分布转化为 $q(\boldsymbol f)$ 在输入 $\mathbf X$ 的潜在函数值上的分布.

这里我们使用 `VariationalStrategy` with `learn_inducing_points=True`，以及 `CholeskyVariationalDistribution`。这些是最直接和常见的选择。

#### The GP model

`ApproximateGP` 模型是gpytorch的最简单的近似推断模型，他用 `VariationalDistribution` 指定的分布来近似真实后验，通常是某种形式的多变量高斯分布。该模型定义了所需的所有变分参数，并将这些信息都隐藏起来。

`ApproximateGP` 模型组件：
- __init__: 它构造一个平均模块 (mean module)，一个内核模块 (kernel module)，一个变分分布对象 (variational distribution) 和一个变分策略对象 (variational strategy)。该方法还应该负责构造可能需要的任何其他模块。
- forward: 输入 $n \times d$ 的数据 $\mathbf X$ 并返回一个 MultivariatNormal，其中包含在x处的 先验均值 和 协方差。即返回向量 $\mu(x)$ 和 $n \times n$ 的矩阵 $\mathbf K_{XX}$ 表示GP的先验均值和协方差矩阵。

In [5]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy


class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution,
                                                   learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


inducing_points = train_x[:500, :]  # 训练集的前500作为诱导点输入
model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()  # 高斯似然

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# if torch.cuda.is_available():
#     model = model.cuda()
#     likelihood = likelihood.cuda()
model = model.to(device)
likelihood = likelihood.to(device)

#### Training the Model

与使用精确的GP边际对数似然不同，执行变分推理允许我们使用随机优化技术。对于这个例子，我们将进行一个epoch的训练。考虑到神经网络相对于数据集的大小较小，这应该足以达到与DKL论文中观察到的相当的准确性。

优化循环与我们在更简单的教程中看到的循环不同，因为它涉及到对许多训练迭代(epoch)和小批量数据的循环。然而，基本过程是相同的:对于每个小批量，我们向前通过模型，计算损失(VariationalELBO或ELBO)，向后调用，并执行优化步骤。

In [6]:
# from tqdm.notebook import tqdm
from tqdm import tqdm

num_epochs = 1 if smoke_test else 4

model.train()
likelihood.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

# Our loss object. We're using the VariationalELBO (marginal log likelihood --> ELBO)
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

epochs_iter = tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    # Within each iteration, we will go over each minibatch of data
    minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        optimizer.zero_grad()
        output = model(x_batch)
        loss = -mll(output, y_batch)
        minibatch_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()

#### Making Predictions

In [7]:
model.eval()
likelihood.eval()
means = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])
means = means[1:]

In [8]:
print('Test MAE: {}'.format(torch.mean(torch.abs(means - test_y.cpu()))))

## Modifying the Variational Strategy/Variational Distribution

近似GP的预测分布：
$$\begin{aligned}
    p(\boldsymbol f(\boldsymbol x_*)) = \int_{\boldsymbol u} p(\boldsymbol f(\boldsymbol x_*) | \boldsymbol u)\, q(\boldsymbol u)\, \mathrm{d}\boldsymbol u,\quad q(\boldsymbol u) = \mathcal N(\boldsymbol m, \mathbf S)
\end{aligned}$$
$\boldsymbol u = \boldsymbol f_Z$ 代表 $m$ 个诱导点处的函数值。$\boldsymbol m \in \mathbb R^m, \mathbf S \in \mathbb R^{m \times m}$ 为可学习参数。 

如果 $m$ (诱导点的数量)相当大，可学习参数的数量在 $\mathbf S$ 会很笨重。此外，一个巨大的 $m$ 可能会使一些计算变得很慢。这里我们展示了几种使用不同的变分分布和变分策略来实现这一目标的方法。

### Experimental Setup

In [6]:
import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

import urllib.request
import os
from scipy.io import loadmat
from math import floor
from tqdm.notebook import tqdm

# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)

if not smoke_test and not os.path.isfile('./datasets/elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk',
                               './datasets/elevators.mat')  # 这里 mat 文件需要手动从链接中下载

if smoke_test:  # this is for running the notebook in our testing framework
    X, y = torch.randn(1000, 3), torch.randn(1000)
else:
    data = torch.Tensor(loadmat('./datasets/elevators.mat')['data'])
    X = data[:, :-1]
    X = X - X.min(0)[0]
    X = 2 * (X / X.max(0)[0]) - 1
    y = data[:, -1]

train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
# if torch.cuda.is_available():
# train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()
train_x, train_y, test_x, test_y = train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)

from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)  # original: batch_size=1024

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

### Some Quick Training/Testing Code

In [7]:
# this is for running the notebook in our testing framework
num_epochs = 1 if smoke_test else 10


# Our testing script takes in a GPyTorch MLL (objective function) class
# and then trains/tests an approximate GP with it on the supplied dataset

def train_and_test_approximate_gp(model_cls):
    inducing_points = torch.randn(128, train_x.size(-1), dtype=train_x.dtype, device=train_x.device)
    model = model_cls(inducing_points)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.numel())
    optimizer = torch.optim.Adam(list(model.parameters()) + list(likelihood.parameters()), lr=0.1)

    # if torch.cuda.is_available():
    #     model = model.cuda()
    #     likelihood = likelihood.cuda()
    model = model.to(device)
    likelihood = likelihood.to(device)

    # Training
    model.train()
    likelihood.train()
    epochs_iter = tqdm(range(num_epochs), desc=f"Training {model_cls.__name__}")
    for i in epochs_iter:
        # Within each iteration, we will go over each minibatch of data
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = -mll(output, y_batch)
            epochs_iter.set_postfix(loss=loss.item())
            loss.backward()
            optimizer.step()

    # Testing
    model.eval()
    likelihood.eval()
    means = torch.tensor([0.])
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            preds = model(x_batch)
            means = torch.cat([means, preds.mean.cpu()])
    means = means[1:]
    error = torch.mean(torch.abs(means - test_y.cpu()))
    print(f"Test {model_cls.__name__} MAE: {error.item()}")

#### The Standard Approach

默认情况下，我们将使用带有 ChooleskyVariationalDistribution 的默认 VariationalStrategy 类。`CholeskyVariationalDistribution` 类允许在任何正半定矩阵 $\mathbf S$ 上。这是近似GP的最普遍/最具表现力的选择。

In [8]:
class StandardApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.CholeskyVariationalDistribution(inducing_points.size(-2))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [9]:
train_and_test_approximate_gp(StandardApproximateGP)

### Reducing Parameters

#### MeanFieldVariationalDistribution: a diagonal $\mathbf S$ matrix

减少参数数量的一种方法是对其进行限制：令 $\mathbf S$ 是对角矩阵。这不是很有表现力，但是参数的数量现在是 $m$ 的线性的而不是二次。

将 `CholeskyVariationalDistribution` (full $\mathbf S$ 矩阵) 改成 `MeanFieldVariationalDistribution` (diagonal $\mathbf S$ 矩阵)

In [11]:
class MeanFieldApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.MeanFieldVariationalDistribution(inducing_points.size(-2))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [12]:
train_and_test_approximate_gp(MeanFieldApproximateGP)

#### DeltaVariationalDistribution: no $\mathbf S$ matrix

一种更加极端的减少参数的方法是完全去掉 $\mathbf S$ 矩阵。这对应于学习一个 delta distribution ($\boldsymbol u = \boldsymbol m$) 而不是 $\boldsymbol u$ 的多变量的高斯分布 (multivariate normal distribution)。换句话说，这对应于 **执行MAP估计而不是变分推理**。

In [13]:
class MAPApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.DeltaVariationalDistribution(inducing_points.size(-2))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [14]:
train_and_test_approximate_gp(MAPApproximateGP)

#### Reducing computation (through decoupled inducing points)

降低计算复杂度的一种方法是分别对均值和协方差进行诱导点计算。

在GPyTorch中，我们以模块化的方式实现此方法。正交解耦变分策略 OrthogonallyDecoupledVariationalStrategy 定义了均值诱导点的变分策略。它包含了一个定义协方差诱导点的现有变分策略/分布:

In [15]:
def make_orthogonal_vs(model, train_x):
    mean_inducing_points = torch.randn(1000, train_x.size(-1), dtype=train_x.dtype, device=train_x.device)
    covar_inducing_points = torch.randn(100, train_x.size(-1), dtype=train_x.dtype, device=train_x.device)

    covar_variational_strategy = gpytorch.variational.VariationalStrategy(
        model, covar_inducing_points,
        gpytorch.variational.CholeskyVariationalDistribution(covar_inducing_points.size(-2)),
        learn_inducing_locations=True
    )

    variational_strategy = gpytorch.variational.OrthogonallyDecoupledVariationalStrategy(
        covar_variational_strategy, mean_inducing_points,
        gpytorch.variational.DeltaVariationalDistribution(mean_inducing_points.size(-2)),
    )
    return variational_strategy

In [17]:
class OrthDecoupledApproximateGP(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.DeltaVariationalDistribution(inducing_points.size(-2))
        variational_strategy = make_orthogonal_vs(self, train_x)
        super().__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


train_and_test_approximate_gp(OrthDecoupledApproximateGP)

## Using Natural Gradient Descent with Variational Models

### Overview

优化变分GPyTorch模型时使用自然梯度下降。这将与SVGP回归笔记本相同，只是我们将使用不同的优化器。

### What is natural gradient descent (NGD)?

使用SGD或Adam并不是优化变分高斯分布参数的最佳方法。本质上，SGD采取的步骤假设参数的损失几何是欧几里得的。对于许多分布的参数来说，这是一个糟糕的假设，尤其是高斯分布。请参阅[Agustinus Kristiadi的博客文章](https://wiseodd.github.io/techblog/2018/03/14/natural-gradient/)了解更多细节。

相反，采用**更适合变分分布参数几何形状**的梯度步骤更有意义。具体来说，如果 $boldsymbol m$ 和 $\mathbf S$ 是变分高斯后验近似的均值和协方差，那么如果我们采取以下步骤，我们将会实现更快收敛：
$$
    \begin{bmatrix}
        \boldsymbol m \\
        \mathbf S \\
    \end{bmatrix} \leftarrow
    \begin{bmatrix}
        \boldsymbol m \\
        \mathbf S \\
    \end{bmatrix} - \alpha \mathbf F^{-1} \nabla 
    \begin{bmatrix}
        \boldsymbol m \\
        \mathbf S
    \end{bmatrix}
$$

其中 $\alpha$ 是步长，$\mathbf F$ 是这个分布对应的 **Fisher information matrix** (费雪信息矩阵)。这被称为自然梯度下降 NGD。

事实证明，对于高斯分布(更广泛地说，对于指数族中的所有分布)，存在有效的NGD更新方程。有关更多信息，请参阅以下文件: [Salimbeni, Hugh, Stefanos Eleftheriadis, and James Hensman. “Natural gradients in practice: Non-conjugate variational inference in gaussian process models.” AISTATS (2018). - Hensman, James, Magnus Rattray, and Neil D. Lawrence. “Fast variational inference in the conjugate exponential family.” NeurIPS (2012).](https://arxiv.org/abs/1803.09151)

### Jointly optimizing variational parameters/hyperparameters

In [1]:
# import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

import urllib.request
import os
from scipy.io import loadmat
from math import floor
from tqdm.notebook import tqdm

# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)

if not smoke_test and not os.path.isfile('./datasets/elevators.mat'):
    print('Downloading \'elevators\' UCI dataset...')
    urllib.request.urlretrieve('https://drive.google.com/uc?export=download&id=1jhWL3YUHvXIaftia4qeAyDwVxo6j1alk',
                               './datasets/elevators.mat')  # 这里 mat 文件需要手动从链接中下载

if smoke_test:  # this is for running the notebook in our testing framework
    X, y = torch.randn(1000, 3), torch.randn(1000)
else:
    data = torch.Tensor(loadmat('./datasets/elevators.mat')['data'])
    X = data[:, :-1]
    X = X - X.min(0)[0]
    X = 2 * (X / X.max(0)[0]) - 1
    y = data[:, -1]

train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
# if torch.cuda.is_available():
# train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()
train_x, train_y, test_x, test_y = train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)

from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=512, shuffle=True)  # original: batch_size=1024

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=512, shuffle=False)

### SVGP models for NGD

NGD-SVGP模型与来自SVGP回归笔记本的标准SVGP模型之间有三个关键区别。

#### Difference 1: NaturalVariationalDistribution

不是使用gpytorch.variation.CholeskyVarationalDistribution(或其他变分分布对象)，你必须使用以下两个对象之一:
- `gpytorch.variational.NaturalVariationalDistribution` 通常更快的优化收敛，但对非共轭似然不太稳定
- `gpytorch.variational.TrilNaturalVariationalDistribution` 通常较慢的优化收敛，但对非共轭似然更稳定

In [2]:
class GPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.NaturalVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.VariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


inducing_points = train_x[:500, :]
model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# if torch.cuda.is_available():
#     model = model.cuda()
#     likelihood = likelihood.cuda()
model = model.to(device)
likelihood = likelihood.to(device)

#### Difference 2: Two optimizers - one for the variational parameters; one for the hyperparameters

NGD步骤只更新变分参数。因此，我们需要两个独立的优化器:一个用于变分参数(使用NGD)，另一个用于其他超参数(使用Adam或任何您想要的)。

关于NGD变分优化器需要注意的一些事项:
- 你必须使用gpytorch.optim。作为变分NGD优化器!自适应梯度算法会打乱自然梯度步骤。(任何随机优化器都适用于超参数。)
- 对变分优化器使用较大的学习率。一般来说，0.1是一个很好的学习率。

In [3]:
variational_ngd_optimizer = gpytorch.optim.NGD(model.variational_parameters(), num_data=train_y.size(0), lr=0.1)

hyperparameter_optimizer = torch.optim.Adam([
    {'params': model.hyperparameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

#### Difference 3: The updated training loop

在训练循环中，我们必须更新两个优化器:
- `variational_ngd_optimizer`
- `hyperparameter_optimizer`

In [4]:
model.train()
likelihood.train()
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

num_epochs = 1 if smoke_test else 4
epochs_iter = tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)
    # minibatch_iter = tqdm(train_loader, desc="Minibatch", )

    for x_batch, y_batch in minibatch_iter:
        ### Perform NGD step to optimize variational parameters
        variational_ngd_optimizer.zero_grad()
        hyperparameter_optimizer.zero_grad()
        output = model(x_batch)
        loss = -mll(output, y_batch)
        minibatch_iter.set_postfix(loss=loss.item())
        loss.backward()
        variational_ngd_optimizer.step()  # 更新 变分优化器
        hyperparameter_optimizer.step()  # 更新 超参优化器

你也可以修改优化循环，让它在NGD步骤和超参数更新之间交替进行:

In [5]:
for x_batch, y_batch in minibatch_iter:
    ### Perform NGD step to optimize variational parameters
    variational_ngd_optimizer.zero_grad()
    output = model(x_batch)
    loss = -mll(output, y_batch)
    minibatch_iter.set_postfix(loss=loss.item())
    loss.backward()
    variational_ngd_optimizer.step()

    ### Perform Adam step to optimize hyperparameters
    # 交替优化
    hyperparameter_optimizer.zero_grad()
    output = model(x_batch)
    loss = -mll(output, y_batch)
    loss.backward()
    hyperparameter_optimizer.step()

#### Evaluation

这个模型应该比标准的SVGP模型收敛得更快/更好——而且它通常可以获得更好的性能(特别是当使用更多的诱导点时)。

In [6]:
model.eval()
likelihood.eval()
means = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])
means = means[1:]
print('Test MAE: {}'.format(torch.mean(torch.abs(means - test_y.cpu()))))

## Stochastic Variational GP Regression with Contour Integral Quadrature

具有等值积分正交的随机变分GP回归

### Overview

如何使用Pleiss等人，2020中描述的轮廓积分正交(CIQ)和msMINRES进行随机变分GP回归。在下列情况下，可以用轮廓积分正交代替标准SVGP:
- 诱导点很多(如M > 5000)
- 诱导点具有特殊的结构(如位于网格上)。

我们将概述如何使用[CIQ-SVGP随机变分回归](https://arxiv.org/pdf/1411.2005.pdf)在3droad UCI数据集上使用minibatch快速训练。

In [20]:
# import tqdm
import math
import torch
import gpytorch
from matplotlib import pyplot as plt

import urllib.request
import os
from scipy.io import loadmat
from math import floor
from tqdm.notebook import tqdm

# this is for running the notebook in our testing framework
smoke_test = ('CI' in os.environ)

if not smoke_test and not os.path.isfile('./datasets/3droad.mat'):
    print('Downloading \'3droad\' UCI dataset...')
    urllib.request.urlretrieve('https://www.dropbox.com/s/f6ow1i59oqx05pl/3droad.mat?dl=1', './datasets/3droad.mat')  # 这里 mat 文件需要手动从链接中下载

if smoke_test:  # this is for running the notebook in our testing framework
    X, y = torch.randn(10, 2), torch.randn(10)
else:
    data = torch.Tensor(loadmat('./datasets/3droad.mat')['data'])
    X = data[:, :-2]
    X = X - X.min(0)[0]
    X = 2 * (X / X.max(0)[0]) - 1
    y = data[:, -1]
    y.sub_(y.mean(0)).div_(y.std(0))
    
    # Let's subsample the data
    indices = torch.randperm(X.size(0))[:10000]
    X = X[indices]
    y = y[indices]

train_n = int(floor(0.8 * len(X)))
train_x = X[:train_n, :].contiguous()
train_y = y[:train_n].contiguous()

test_x = X[train_n:, :].contiguous()
test_y = y[train_n:].contiguous()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
# if torch.cuda.is_available():
# train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()
train_x, train_y, test_x, test_y = train_x.to(device), train_y.to(device), test_x.to(device), test_y.to(device)


### DataLoaders with CIQ-SVGP

CIQ仅在小批量大小远小于诱导点数量时提供计算加速。我们发现256的小批量通常效果很好。

In [21]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
# Smaller batch sizes are better for CIQ

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

### Number of inducing points

当有很多诱导点时，CIQ提供了加速计算。这里，我们选择2000个诱导点。

In [22]:
inducing_points = train_x[torch.randperm(train_x.size(0))[:2000]]

### CIQ - SVGP models

要使用轮廓积分正交，只需将 `VariationalStrategy` 替换为 `CiqVariationalStrategy`。

在这个例子中，我们使用的是 `NaturalVariationalStrategy`，因为CIQ在自然梯度下降的情况下效果最好。

In [23]:
class GPModel(gpytorch.models.ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = gpytorch.variational.NaturalVariationalDistribution(inducing_points.size(0))
        variational_strategy = gpytorch.variational.CiqVariationalStrategy(
            self, inducing_points, variational_distribution, learn_inducing_locations=True
        )
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=2)
        )
        self.covar_module.base_kernel.initialize(lengthscale=0.01)  # Specific to the 3droad dataset

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

# if torch.cuda.is_available():
#     model = model.cuda()
#     likelihood = likelihood.cuda()
model = model.to(device)
likelihood = likelihood.to(device)

In [24]:
variational_ngd_optimizer = gpytorch.optim.NGD(model.variational_parameters(), num_data=train_y.size(0), lr=0.01)

hyperparameter_optimizer = torch.optim.Adam([
    {'params': model.hyperparameters()},
    {'params': likelihood.parameters()},
], lr=0.01)

In [25]:
model.train()
likelihood.train()
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_y.size(0))

num_epochs = 1 if smoke_test else 4
epochs_iter = tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    minibatch_iter = tqdm(train_loader, desc="Minibatch", leave=False)

    for x_batch, y_batch in minibatch_iter:
        variational_ngd_optimizer.zero_grad()
        hyperparameter_optimizer.zero_grad()
        output = model(x_batch)
        loss = -mll(output, y_batch)
        minibatch_iter.set_postfix(loss=loss.item())
        loss.backward()
        variational_ngd_optimizer.step()
        hyperparameter_optimizer.step()

In [26]:
model.eval()
likelihood.eval()
means = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])
means = means[1:]
print('Test MAE: {}'.format(torch.mean(torch.abs(means - test_y.cpu()))))